# Loop 3 Analysis: Survival Rate Hypothesis & Threshold Tuning

## Current Situation
- **Best LB**: 0.7775 (Simple RF, 7 features, 31.3% survival rate)
- **Latest exp_002**: Voting Ensemble, CV 0.8328, 37.6% survival rate - NOT SUBMITTED
- **Target**: 1.0 (IMPOSSIBLE - state-of-the-art is 81-85%)

## Key Pattern Discovered
| Model | CV | LB | Survival Rate |
|-------|-----|-----|---------------|
| XGBoost (13 features) | 0.8316 | 0.7584 | 37.6% |
| Simple RF (7 features) | 0.8238 | 0.7775 | 31.3% |
| Voting Ensemble (8 features) | 0.8328 | ? | 37.6% |

**Hypothesis**: Lower survival rate predictions → Better LB performance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load all submissions
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')

sub_baseline = pd.read_csv('/home/code/submission_candidates/candidate_000.csv')
sub_simple = pd.read_csv('/home/code/submission_candidates/candidate_001.csv')
sub_ensemble = pd.read_csv('/home/code/submission_candidates/candidate_002.csv')

print("Survival Rate Comparison:")
print(f"Training data: {train['Survived'].mean():.3f} ({train['Survived'].sum()}/{len(train)})")
print(f"\nSubmissions:")
print(f"  exp_000 (XGBoost):  {sub_baseline['Survived'].mean():.3f} ({sub_baseline['Survived'].sum()}/{len(sub_baseline)}) → LB 0.7584")
print(f"  exp_001 (Simple RF): {sub_simple['Survived'].mean():.3f} ({sub_simple['Survived'].sum()}/{len(sub_simple)}) → LB 0.7775")
print(f"  exp_002 (Ensemble):  {sub_ensemble['Survived'].mean():.3f} ({sub_ensemble['Survived'].sum()}/{len(sub_ensemble)}) → LB ???")

In [ ]:
# Compare predictions between models
print("\nPrediction Agreement Analysis:")
print(f"XGBoost vs Simple RF: {(sub_baseline['Survived'] == sub_simple['Survived']).mean():.1%}")
print(f"XGBoost vs Ensemble: {(sub_baseline['Survived'] == sub_ensemble['Survived']).mean():.1%}")
print(f"Simple RF vs Ensemble: {(sub_simple['Survived'] == sub_ensemble['Survived']).mean():.1%}")

# The ensemble is likely very similar to XGBoost
print(f"\nEnsemble predicts same as XGBoost: {(sub_baseline['Survived'] == sub_ensemble['Survived']).sum()}/418")
print(f"Ensemble predicts same as Simple RF: {(sub_simple['Survived'] == sub_ensemble['Survived']).sum()}/418")

In [ ]:
# Analyze where Simple RF differs from Ensemble
diff_mask = sub_simple['Survived'] != sub_ensemble['Survived']
diff_ids = sub_simple[diff_mask]['PassengerId'].values
diff_test = test[test['PassengerId'].isin(diff_ids)].copy()

print(f"\nPassengers where Simple RF ≠ Ensemble: {len(diff_test)}")
print(f"\nSex distribution:")
print(diff_test['Sex'].value_counts())
print(f"\nPclass distribution:")
print(diff_test['Pclass'].value_counts())

# What predictions differ?
rf_preds = sub_simple[diff_mask]['Survived'].values
ens_preds = sub_ensemble[diff_mask]['Survived'].values
print(f"\nSimple RF predicts survivors: {rf_preds.sum()}")
print(f"Ensemble predicts survivors: {ens_preds.sum()}")

In [ ]:
# KEY INSIGHT: If Simple RF (31.3% survival) beat XGBoost (37.6% survival),
# then Ensemble (37.6% survival) will likely perform similar to XGBoost

print("="*60)
print("SURVIVAL RATE HYPOTHESIS")
print("="*60)
print(f"""
The pattern is clear:
- Training survival rate: 38.4%
- XGBoost predicted: 37.6% → LB 0.7584 (close to train rate)
- Simple RF predicted: 31.3% → LB 0.7775 (lower than train rate)
- Ensemble predicted: 37.6% → LB ??? (same as XGBoost)

This suggests the TEST SET has a LOWER survival rate than training.
Models that predict fewer survivors perform better on LB.

Prediction: Ensemble will score ~0.76-0.77 on LB (similar to XGBoost)
""")

# Calculate expected LB based on survival rate pattern
print("\nExpected LB based on survival rate pattern:")
print(f"  If survival rate matters: ~0.76-0.77 (similar to XGBoost)")
print(f"  If CV matters: ~0.78-0.79 (CV 0.8328 - 5% gap)")

In [ ]:
# THRESHOLD TUNING ANALYSIS
# The evaluator suggested adjusting threshold from 0.5 to 0.55-0.60
# Let's see what survival rates we'd get with different thresholds

# Load the ensemble probabilities (need to re-run or load from notebook)
# For now, let's analyze what threshold would give us ~31% survival rate

print("="*60)
print("THRESHOLD TUNING STRATEGY")
print("="*60)
print(f"""
Current ensemble uses threshold 0.5 → 37.6% survival rate
Simple RF achieved 31.3% survival rate → LB 0.7775

To match Simple RF's survival rate, we need to:
1. Increase threshold to ~0.55-0.60
2. This would reduce predicted survivors from 157 to ~130

Expected outcome:
- If survival rate hypothesis is correct: LB should improve to ~0.77-0.78
- If CV is more important: LB might decrease

This is a testable hypothesis with our remaining submissions.
""")

# Calculate what threshold would give ~31% survival rate
target_survivors = int(418 * 0.313)  # ~131 survivors
current_survivors = sub_ensemble['Survived'].sum()
print(f"\nCurrent survivors: {current_survivors}")
print(f"Target survivors (31.3%): {target_survivors}")
print(f"Need to reduce by: {current_survivors - target_survivors} predictions")

In [ ]:
# STRATEGIC DECISION
print("="*60)
print("STRATEGIC DECISION")
print("="*60)
print(f"""
Option 1: Submit exp_002 (Ensemble) as-is
- Expected LB: ~0.76-0.77 (based on survival rate pattern)
- Purpose: Validate that survival rate matters more than CV
- Risk: Wastes a submission if hypothesis is correct

Option 2: Create threshold-adjusted version first
- Adjust threshold to get ~31% survival rate
- Expected LB: ~0.77-0.78 (if hypothesis correct)
- Purpose: Test if threshold tuning improves LB

Option 3: Try stacking approach
- Use out-of-fold predictions as meta-features
- Research shows this achieved 0.808 LB
- More complex but potentially higher ceiling

RECOMMENDATION:
1. First, submit exp_002 to validate survival rate hypothesis
2. Then, create threshold-adjusted version and submit
3. If both confirm pattern, focus on calibration over CV
4. Consider stacking for final push
""")

print(f"\nRemaining submissions: 6")
print(f"Current best LB: 0.7775")
print(f"Target: 1.0 (IMPOSSIBLE - realistic target is 0.80-0.81)")

## Summary

### Key Findings
1. **Survival rate pattern**: Lower predicted survival rate → Better LB
2. **Ensemble (exp_002)**: Has same survival rate as XGBoost (37.6%), likely similar LB
3. **Threshold tuning**: Could reduce survival rate to match Simple RF's 31.3%

### Evaluator's Feedback
- Technical execution: TRUSTWORTHY
- Key concern: Survival rate (37.6%) matches XGBoost, not Simple RF
- Suggestion: Adjust threshold from 0.5 to 0.55-0.60
- Priority: Submit current model AND threshold-adjusted version

### Next Steps
1. **Submit exp_002** to validate survival rate hypothesis
2. **Create threshold-adjusted version** with ~31% survival rate
3. **Consider stacking** for higher ceiling (0.808 LB reported)
4. **Focus on calibration** rather than CV optimization